---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [27]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from datetime import datetime

In [64]:
def blight_model():
    
    # Your code here
    train = pd.read_csv('train.csv', engine='python')

    addr = pd.read_csv('addresses.csv')

    latlon = pd.read_csv('latlons.csv')

    result = pd.merge(addr,latlon,how='left',on=['address'])
    result.drop(['address'], axis=1, inplace=True)

    train = pd.merge(train,result,how='left',on=['ticket_id'])

    test = pd.read_csv('test.csv')
    test = pd.merge(test,result,how='left',on='ticket_id')

    train.set_index('ticket_id')
    test.set_index('ticket_id')

    train = train[(train['compliance'] == 0) | (train['compliance'] == 1)]

    train['hearing_date'] = pd.to_datetime(train['hearing_date'])
    train['ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'])

    train['no_days'] = (train['hearing_date']-train['ticket_issued_date']).dt.days
    train.columns

    test['hearing_date'] = pd.to_datetime(test['hearing_date'])
    test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])

    test['no_days'] = (test['hearing_date']-test['ticket_issued_date']).dt.days
    test.columns

    remove_train = ['payment_status','payment_date', 'payment_amount',
                       'balance_due','collection_status',
                       'compliance_detail']

    remove_both = ['fine_amount', 'violator_name', 'zip_code',
                   'country', 'city','inspector_name',
                   'violation_street_number',
                   'violation_street_name',
                   'violation_zip_code',
                   'violation_description',
                   'mailing_address_str_number', 
                   'mailing_address_str_name',
                   'non_us_str_code','ticket_issued_date',
                   'hearing_date', 'grafitti_status',
                   'violation_code','state','disposition','agency_name']

    train.drop(remove_train, axis=1, inplace=True)
    train.drop(remove_both, axis=1, inplace=True)
    test.drop(remove_both, axis=1, inplace=True)

    ticket_id_series = test['ticket_id']

    train.set_index('ticket_id', inplace=True)
    test.set_index('ticket_id', inplace=True)

    ytrain = train['compliance']
    train.drop(['compliance'], axis=1, inplace=True)
    xtrain = train
    xtest = test

    xtrain.lat.fillna(method='pad', inplace=True)
    xtrain.lon.fillna(method='pad', inplace=True)
    xtrain.no_days.fillna(method='pad', inplace=True)
    
    xtest.lat.fillna(method='pad', inplace=True)
    xtest.lon.fillna(method='pad', inplace=True)
    xtest.no_days.fillna(method='pad', inplace=True)
    
    scaler = MinMaxScaler()
    xtrain_scale = scaler.fit_transform(xtrain)
    xtest_scale = scaler.transform(xtest)

    xtrain_copy = np.nan_to_num(xtrain_scale)
    xtest_copy = np.nan_to_num(xtest_scale)
    
    clf = MLPClassifier(hidden_layer_sizes = [100, 11],
                    alpha=0.001,
                    random_state = 0, 
                    solver='lbfgs')
    clf.fit(xtrain_copy, ytrain)
    
    y_pred = clf.predict_proba(xtest_copy)[:,1]
    out = pd.Series(y_pred, index=ticket_id_series)
    
    return out
# blight_model()

ticket_id
284932    0.047812
285362    0.030321
285361    0.065379
285338    0.057344
285346    0.087127
285345    0.056997
285347    0.126063
285342    0.344066
285530    0.037286
284989    0.028176
285344    0.124601
285343    0.032726
285340    0.033518
285341    0.126083
285349    0.087171
285348    0.057027
284991    0.028167
285532    0.031869
285406    0.025230
285001    0.027965
285006    0.031945
285405    0.030454
285337    0.024775
285496    0.097728
285497    0.048555
285378    0.029745
285589    0.025673
285585    0.042927
285501    0.075101
285581    0.029260
            ...   
376367    0.024113
376366    0.036507
376362    0.034339
376363    0.042773
376365    0.024113
376364    0.036507
376228    0.044043
376265    0.037032
376286    0.316684
376320    0.041172
376314    0.041226
376327    0.359164
376385    0.331614
376435    0.523108
376370    0.361934
376434    0.085875
376459    0.081049
376478    0.017340
376473    0.041484
376484    0.035212
376482    0.022839
37

In [63]:
# xtest.isnull().any()

admin_fee          False
state_fee          False
late_fee           False
discount_amount    False
clean_up_cost      False
judgment_amount    False
lat                 True
lon                 True
no_days             True
dtype: bool

In [55]:
# train = pd.read_csv('train.csv', engine='python')

# addr = pd.read_csv('addresses.csv')
# addr.head()

# latlon = pd.read_csv('latlons.csv')
# latlon.head()

# result = pd.merge(addr,latlon,how='left',on=['address'])
# result.drop(['address'], axis=1, inplace=True)
# result.head()

# train = pd.merge(train,result,how='left',on=['ticket_id'])
# train.head()

# test = pd.read_csv('test.csv')
# test.head()

# test = pd.merge(test,result,how='left',on='ticket_id')
# test.head()

# train.set_index('ticket_id')
# test.set_index('ticket_id')

# train = train[(train['compliance'] == 0) | (train['compliance'] == 1)]

# train['hearing_date'] = pd.to_datetime(train['hearing_date'])
# train['ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'])

# train['no_days'] = (train['hearing_date']-train['ticket_issued_date']).dt.days
# train.columns

# test['hearing_date'] = pd.to_datetime(test['hearing_date'])
# test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])

# test['no_days'] = (test['hearing_date']-test['ticket_issued_date']).dt.days
# test.columns

# remove_train = ['payment_status','payment_date', 'payment_amount',
#                    'balance_due','collection_status',
#                    'compliance_detail']

# remove_both = ['fine_amount', 'violator_name', 'zip_code',
#                'country', 'city','inspector_name',
#                'violation_street_number',
#                'violation_street_name',
#                'violation_zip_code',
#                'violation_description',
#                'mailing_address_str_number', 
#                'mailing_address_str_name',
#                'non_us_str_code','ticket_issued_date',
#                'hearing_date', 'grafitti_status',
#                'violation_code','state','disposition','agency_name']

# train.drop(remove_train, axis=1, inplace=True)
# train.drop(remove_both, axis=1, inplace=True)
# test.drop(remove_both, axis=1, inplace=True)

# ticket_id_series = test['ticket_id']

# train.set_index('ticket_id', inplace=True)
# test.set_index('ticket_id', inplace=True)

# ytrain = train['compliance']
# train.drop(['compliance'], axis=1, inplace=True)
# xtrain = train
# xtest = test

# scaler = MinMaxScaler()
# xtrain_scale = scaler.fit_transform(xtrain)
# xtest_scale = scaler.fit_transform(xtest)

# xtrain_copy = np.nan_to_num(xtrain_scale)
# xtest_copy = np.nan_to_num(xtest_scale)

In [51]:
# train

,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,lat,lon,no_days
ticket_id,,,,,,,,,
22056,20.0,10.0,25.0,0.0,0.0,305.0,42.390729,-83.124268,369.0
27586,20.0,10.0,75.0,0.0,0.0,855.0,42.326937,-83.135118,378.0
22046,20.0,10.0,25.0,0.0,0.0,305.0,42.145257,-83.208233,323.0
18738,20.0,10.0,75.0,0.0,0.0,855.0,42.433466,-83.023493,253.0
18735,20.0,10.0,10.0,0.0,0.0,140.0,42.388641,-83.037858,251.0
18733,20.0,10.0,10.0,0.0,0.0,140.0,42.388641,-83.037858,251.0
28204,20.0,10.0,75.0,0.0,0.0,855.0,42.435773,-82.963348,323.0
18743,20.0,10.0,75.0,0.0,0.0,855.0,42.395765,-83.022333,209.0
18741,20.0,10.0,75.0,0.0,0.0,855.0,42.440190,-83.180488,201.0


In [56]:
# clf = MLPClassifier(hidden_layer_sizes = [100, 11],
#                     alpha=0.001,
#                     random_state = 0, 
#                     solver='lbfgs')
# clf.fit(xtrain_copy, ytrain)

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[100, 11], learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=0, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [57]:
# y_pred = clf.predict_proba(xtest_copy)[:,1]
# out = pd.Series(y_pred, index=ticket_id_series)
# out

ticket_id
284932    3.057017e-05
285362    5.488833e-06
285361    2.821327e-05
285338    1.048855e-04
285346    1.695969e-04
285345    9.535638e-05
285347    2.478294e-04
285342    1.065552e-03
285530    2.194321e-05
284989    3.862787e-05
285344    2.247821e-04
285343    1.528351e-05
285340    1.617839e-05
285341    2.533920e-04
285349    1.715163e-04
285348    9.643565e-05
284991    3.859483e-05
285532    4.875696e-05
285406    2.055868e-05
285001    4.018141e-05
285006    1.744886e-05
285405    4.906573e-06
285337    1.536929e-05
285496    7.526218e-05
285497    3.685755e-05
285378    5.067702e-06
285589    1.626987e-05
285585    2.634861e-05
285501    6.051538e-05
285581    3.696870e-06
              ...     
376367    1.418114e-05
376366    2.842602e-05
376362    1.742212e-05
376363    1.819375e-05
376365    1.418114e-05
376364    2.842602e-05
376228    5.215035e-05
376265    4.736002e-05
376286    2.646884e-04
376320    3.619480e-05
376314    2.647496e-05
376327    9.258524e-04
3